<a href="https://colab.research.google.com/github/907Resident/caldera-gas-emissions/blob/logistic-prediction/forecast_sat_concentration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Saturation Concentration
In this exercise, the gas measurements made at the various locations around Yellowstone and Valles Caldera are examined as time series. From there, I apply a logistic regression to forecast the concentration (and the time) the measurements would have yielded in the field.


## Import Necessary Libraries

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io

# Import fbprphet for forecasting
from fbprophet import Prophet

## Ingest data into workspace
The ultimate goal is to utilze the Microsoft's `onedrivesdk` package to acces data straight from OneDrive using their [Graph API](https://docs.microsoft.com/en-us/graph/api/overview?view=graph-rest-1.0). This is currently issue #1 in the repo. 

For the time being, data is ingested manually using `files` from the `google.colab` library.

In [ ]:
# Manual File Upload from local drive
from google.colab import files
uploaded = files.upload()

In [ ]:
# Import file into workspace
import io
df_2018_YC = pd.read_excel(io.BytesIO(uploaded['YNP_SummaryData.xlsx']), 
                           sheet_name="2018",
                           header=0)